In [16]:
import pandas as pd
from tqdm import tqdm

In [24]:
adj_df = pd.DataFrame(columns =['team', 'year', 'non_fb_o_rating', 'non_fb_d_rating'])
for year in tqdm(range(1996, 2025)):

    urlo = f"https://stats.inpredictable.com/nba/ssnTeamPoss.php?season={year}"
    urld = f"https://stats.inpredictable.com/nba/ssnTeamPoss.php?season={year}&view=def"
    df = pd.read_html(urlo)[0]
    dfd = pd.read_html(urld)[0]
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    dfd.columns = [' '.join(col).strip() for col in dfd.columns.values]


    # Remove '%' and convert to numeric
    df['After Def Rebound %'] = df['After Def Rebound %'].str.rstrip('%').astype('float') / 100
    df['After Made Shot* %'] = df['After Made Shot* %'].str.rstrip('%').astype('float') / 100
    # Remove '%' and convert to numeric
    dfd['After Def Rebound %'] = dfd['After Def Rebound %'].str.rstrip('%').astype('float') / 100
    dfd['After Made Shot* %'] = dfd['After Made Shot* %'].str.rstrip('%').astype('float') / 100

    # Now apply the formula
    df['non_fb_o_rating'] = df.apply(
        lambda row: (
            (row['After Def Rebound points'] * row['After Def Rebound %'] +
            row['After Made Shot* points'] * row['After Made Shot* %']) /
            (row['After Made Shot* %'] + row['After Def Rebound %'])
        ) if pd.notnull(row['After Def Rebound points']) and pd.notnull(row['After Made Shot* points']) else None, 
        axis=1
    )
    # Now apply the formula
    dfd['non_fb_d_rating'] = dfd.apply(
        lambda row: (
            (row['After Def Rebound points'] * row['After Def Rebound %'] +
            row['After Made Shot* points'] * row['After Made Shot* %']) /
            (row['After Made Shot* %'] + row['After Def Rebound %'])
        ) if pd.notnull(row['After Def Rebound points']) and pd.notnull(row['After Made Shot* points']) else None, 
        axis=1
    )

    combined = pd.merge(df, dfd, on = 'Unnamed: 1_level_0 Team', how = 'left')

    combined['year'] = year
    new_df = pd.DataFrame(combined.loc[combined.index[:-2],['Unnamed: 1_level_0 Team', 'year', 'non_fb_o_rating', 'non_fb_d_rating']])
    new_df.rename(columns = {'Unnamed: 1_level_0 Team':'team'}, inplace=True)

    adj_df = pd.concat([adj_df, new_df], axis = 0)

adj_df.head()    

  0%|          | 0/29 [00:00<?, ?it/s]/var/folders/d4/c154mt9s11x5q965j91l06qw0000gn/T/ipykernel_24571/2818798407.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  adj_df = pd.concat([adj_df, new_df], axis = 0)
100%|██████████| 29/29 [01:42<00:00,  3.54s/it]


,team,year,non_fb_o_rating,non_fb_d_rating
0,CHI,1996,1.104070,0.984713
1,UTA,1996,1.074000,1.000465
2,SEA,1996,1.063902,0.981163
3,CHH,1996,1.056092,1.042759
4,DET,1996,1.048140,1.003933


In [27]:
adj_df.shape

(862, 4)

In [28]:
adj_df.to_csv('team_ratings.csv')

In [41]:
def get_relative_rating(df, o_team, d_team, year):

    o_rate = df[(df['team']==o_team) &( df['year'] == year)]['non_fb_o_rating'].values[0]
    d_rate = df[(df['team']==d_team) &( df['year'] == year)]['non_fb_d_rating'].values[0]

    return (o_rate+d_rate)/2

In [42]:
get_relative_rating(adj_df, 'BOS', 'PHI', 2024)

1.1596666666666664